In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Access and recess equation form the Parisian Alfonsines Tables
Hist_table=HTable.read(1373,format='dishas')



In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(8):
    for j in range(34):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')


In [4]:
for i in range(8):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
        # initial value for superatio
        superatio=Hist_table[i][0]
                 
    for j in range(34):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):
            # Initial values for elongation
            elong=Hist_table[0][1][j][0]
         
        # computation of the proportion
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            prop=elong/superatio
            Test_recomp[i][1][j][1]=prop*u.degree
            hist=Hist_table[i][1][j][1]
            Delta_recomp[i][1][j][1]=3600*(hist-prop)


In [5]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(-516.0808823529412, 390.9581590195988)

In [6]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,00 ; 27,00 ; 28,00 ; 29,00 ; 30,00 ; 31,00 ; 32,00 ; 33,00 ; 34
00 ; 01,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
00 ; 02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00 ; 03,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
00 ; 04,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00 ; 05,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
00 ; 06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00 ; 07,0.000000,0.000000,0.000000,-60.000000,0.000000,-1.000000,0.000000,0.000000
00 ; 08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00 ; 09,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
00 ; 10,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Kammad long_TM.csv')
B.to_csv('Kammad long_TM.zip', index=False, compression=compression_opts)